### IMPORTS

In [1]:
import numpy as np

In [2]:
from sklearn.linear_model import LinearRegression

In [3]:
import tensorflow as tf
from tensorflow.python.keras import Sequential, Input, Model
from tensorflow.python.keras.layers import Dense, LSTM, Dropout

In [4]:
import ipynb.fs.full.splitting as splitting
import ipynb.fs.full.misc as misc

In [5]:
from tcn import TCN, tcn_full_summary

In [6]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [7]:
from sklearn.model_selection import GridSearchCV

In [8]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [9]:
from sklearn.pipeline import Pipeline

### LINEAR REGRESSION

In [10]:
def linear_regression(data, settings):
    
    # INSTANTIATE MODEL & SCALER
    model = LinearRegression()
    
    # SCALE & FIT WITH TRAIN DATA
    model.fit(
        data['train']['features'],
        data['train']['labels']
    )
    
    # SCALE & PREDICT WITH TEST DATA
    predictions = model.predict(data['test']['features'])
    
    return model, np.ndarray.flatten(predictions)

### LONG-SHORT TERM MEMORY

In [11]:
def long_short_term(data, settings):
    
    # INSTANTIATE MODEL & SCALER
    model = Sequential()
    
    # TRAIN DATA GENERATOR
    train_generator = splitting.generator(
        data['train'],
        settings['morph'],
        shuffle=True
    )
    
    # LOOP THROUGH LAYERS
    for index, layer in enumerate(settings['layers']):
        
        # LAYER PROPS
        name, params = misc.key_value(layer)
        
        # ADD LAYER
        add_lstm_layer(
            name,
            params,
            model,
            index,
            train_generator[0][0].shape
        )
    
    # COMPILE THE MODEL
    model.compile(
        loss=settings['loss'],
        optimizer=settings['optimizer']
    )
    
    # FIT USING THE TRAIN GENERATOR
    model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=settings['epochs'],
        verbose=0
    )
    
    # TEST DATA GENERATOR
    test_generator = splitting.generator(
        data['test'],
        settings['morph'],
        shuffle=False
    )
    
    # PREDICT USING TEST DATA
    predictions = model.predict(test_generator)
    
    return model, np.ndarray.flatten(predictions)

In [12]:
def add_lstm_layer(name, settings, model, index, shape):
    
    # AVAILABLE LSTM LAYERS
    available = {
        'lstm': LSTM,
        'dropout': Dropout,
        'dense': Dense
    }

    # SELECT THE CORRECT FUNCTION
    func = available[name]

    # INJECT INPUT LAYER DIMENSIONS TO SETTINGS
    if index == 0:
        settings['input_shape'] = (shape[1], shape[2])
        model.add(func(**settings))

    # OTHERWISE, DEFAULT TO BASE SETTINGS
    else:
        model.add(func(**settings))

### TEMPORAL CONVOLUTIONAL NETWORK

In [13]:
def temporal_convolutional_network(data, settings):

    # TRAIN DATA GENERATOR
    train_generator = splitting.generator(
        data['train'],
        settings['morph'],
        shuffle=True
    )

    # INPUT & OUTPUT LAYER
    model_input = Input(batch_shape=train_generator[0][0].shape)
    model_output = []
    
    # LOOP THROUGH REQUESTED MODEL LAYERS
    for index, layer in enumerate(settings['layers']):

        # LAYER PROPS
        name, params = misc.key_value(layer)
        
        # CHURN MODEL LAYERS
        model_output = add_tcn_layers(
            name,
            params,
            index,
            model_input,
            model_output
        )

    # INSTANTIATE THE MODEL
    model = Model(inputs=[model_input], outputs=[model_output])

    # COMPILE THE MODEL
    model.compile(
        optimizer=settings['optimizer'],
        loss=settings['loss']
    )

    #  PRINT MODEL STATS
    # tcn_full_summary(model, expand_residual_blocks=False)

    # FIT THE MODEL WITH THE TRAIN GENERATOR
    model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=settings['epochs'],
        verbose=0
    )
    
    # TEST DATA GENERATOR
    test_generator = splitting.generator(
        data['test'],
        settings['morph'],
        shuffle=False
    )

    #  PREDICT USING TEST GENERATOR
    predictions = model.predict(test_generator)

    return model, np.ndarray.flatten(predictions)

In [14]:
def add_tcn_layers(name, settings, index, model_input, old_output):
    
    # NEW OUTPUT PLACEHOLDER
    new_output = []
    
    # AVAILABLE TNC LAYERS
    available = {
        'tcn': TCN,
        'dense': Dense,
        'dropout': Dropout
    }

    # SELECT THE CORRECT FUNCTION
    func = available[name]

    # INJECT WITH INPUT LAYER
    if index == 0:
        new_output = func(**settings)(model_input)

    # OTHERWISE, INJECT WITH OUTPUT LAYER
    else:
        new_output = func(**settings)(old_output)
        
    return new_output

### GRIDSEARCH FUNCTIONS

In [15]:
def search(model, dataset, settings):
    
    # COMBINE TRAIN & TEST FEATURES & LABELS
    features, labels = splitting.grid(dataset)
    
    # GRID SEARCH PARAMS
    grid_model = GridSearchCV(model, settings)
    
    # FIT THE GRID
    grid_model.fit(features, labels)
    
    # RETURN THE BEST OUTCOME
    return grid_model.best_estimator_

In [16]:
def construct_base(model, dataset, settings):
    
    # HAS STATIC PARAMETER
    if 'static' in settings:
        model = model(**settings['static'])
    
    # DOES NOT
    else:
        model = model()
    
    # HAS GRID SEARCH PARAMETERS
    if 'grid_search' in settings:
        model = search(model, dataset, settings['grid_search'])
    
    return model

### SUPPORT VECTOR MACHINE

In [17]:
def support_vector_classifier(dataset, settings):
    
    # INSTANTIATE MODEL & SCALER
    model = construct_base(SVC, dataset, settings)
    
    # FIT ON TRAIN DATASET
    model.fit(
        dataset['train']['features'],
        dataset['train']['labels']
    )
    
    # PREDICT WITH TEST DATASET
    predictions = model.predict(dataset['test']['features'])
    
    return model, np.ndarray.flatten(predictions)

### LOGISTIC REGRESSION

In [18]:
def logistic_regression(dataset, settings):
    
    # INSTANTIATE MODEL & SCALER
    model = construct_base(LogisticRegression, dataset, settings)
    
    # FIT WITH TRAIN DATA
    model.fit(
        dataset['train']['features'],
        dataset['train']['labels']
    )
    
    # PREDICT WITH TEST DATASET
    predictions = model.predict(dataset['test']['features'])
    
    return model, np.ndarray.flatten(predictions)

### START TRAINING A MODEL

In [19]:
def start(dataset, name, settings):
    
    # AVAILABLE MODELS
    available = {
        'linreg': linear_regression,
        'lstm': long_short_term,
        'tcn': temporal_convolutional_network,
        'svc': support_vector_classifier,
        'logreg': logistic_regression
    }
    
    # SELECT THE CORRECT FUNCTION & START
    return available[name](dataset, settings)